In [ ]:
!pip install tensorflow
import matplotlib.pyplot as plt 
from google.colab import drive 
from tensorflow.data import Dataset
import tensorflow.keras as keras 
import numpy as np
from google.colab.patches import cv2_imshow
from PIL import Image
import glob
import tensorflow.keras as keras
image_list = []
drive.mount('/content/Drive')
directory = '/content/Drive/MyDrive/IF_Norte_Vapor_Agua/'

for filename in glob.glob(directory+'/*.jpg'): 
    im=Image.open(filename)
    display(im)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [ ]:
tamanho_da_imagem = (200, 200)
tipo_de_cor = 'grayscale'
train_dataset = keras.utils.image_dataset_from_directory(directory,
                                              labels = None,
                                              color_mode = 'grayscale',
                                              image_size=tamanho_da_imagem)

Found 4547 files belonging to 1 classes.


In [ ]:
mapas_train_images = []
for images in train_dataset:
    for i in range(len(images)):
      mapas_train_images.append(images[i])
      
m_images = np.array(mapas_train_images)
print(m_images.shape)
m_images = m_images.reshape(m_images.shape[0],200,200,)

(4547, 200, 200, 1)


In [ ]:
linhas = 200 #pixels 200x200
colunas = 200
canais = 1 #canais em modo ciza
noise_input_dim = 100 # a dimensão do noise_vector é arbitrária 
img_shape = (linhas, colunas, canais)

In [ ]:
def rede_geradora(img_shape, noise_dim): 
  modelo = keras.models.Sequential()
  modelo.add(keras.layers.Dense(28, input_dim=noise_dim))
  modelo.add(keras.layers.LeakyReLU(alpha=0.01))
  modelo.add(keras.layers.Dense(56))
  modelo.add(keras.layers.LeakyReLU(alpha=0.02))
  modelo.add(keras.layers.Dense(112))
  modelo.add(keras.layers.LeakyReLU(alpha=0.02))
  row,column,channel = img_shape 
  modelo.add(keras.layers.Dense(row*column*channel, activation='tanh'))
  modelo.add(keras.layers.Reshape(img_shape))
  return modelo 

def rede_discriminadora(img_shape): 
  modelo = keras.models.Sequential()
  modelo.add(keras.layers.Flatten(input_shape = img_shape))
  modelo.add(keras.layers.Dense(112))
  modelo.add(keras.layers.LeakyReLU(alpha=0.02))
  modelo.add(keras.layers.Dense(56))
  modelo.add(keras.layers.LeakyReLU(alpha=0.02))
  modelo.add(keras.layers.Dense(28))
  modelo.add(keras.layers.LeakyReLU(alpha=0.01))
  modelo.add(keras.layers.Dense(1, activation='sigmoid'))
  return modelo 


In [ ]:
def VanillaGAN(rede_geradora, rede_discriminativa):
  modelo = keras.models.Sequential()
  modelo.add(rede_geradora)
  modelo.add(rede_discriminativa)
  return modelo

gerador = rede_geradora(img_shape, noise_input_dim)
discriminador = rede_discriminadora(img_shape)
discriminador.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
discriminador.trainable = False
gan = VanillaGAN(gerador, discriminador)
gan.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam())

In [ ]:
perdas = []
acuracias = []
iteration_checkpoints = []

def transform_func(X):
    X = X / 127.5 - 1
    return X

def train(iterations, batch_size, sample_interval):
   X_train = m_images
   
   X_train = X_train.reshape(X_train.shape[0], 200, 200, 1).astype('float32')
   
   X_train = (X_train - 127.5) / 127.5 

   #X_train = Dataset.from_tensor_slices(X_train)

   real = np.ones((batch_size, 1))

   fake = np.zeros((batch_size, 1))

   for iteration in range(iterations):

     idx = np.random.randint(0,X_train.shape[0], batch_size) 
     imgs = X_train[idx] 

     z = np.random.normal(0,1,(batch_size, 100)) 
     gen_imgs = gerador.predict(z)
     
     d_loss_real = discriminador.train_on_batch(imgs, real)
     d_loss_fake = discriminador.train_on_batch(gen_imgs, fake) 
     d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)

     z = np.random.normal(0,1, (batch_size, 100))
     gen_imgs = gerador.predict(z)
     g_loss = gan.train_on_batch(z, real) 
     if (iteration + 1) % sample_interval == 0:
        perdas.append((d_loss, g_loss)) 
        acuracias.append(100.0 * accuracy)
        iteration_checkpoints.append(iteration + 1)
        print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %(iteration + 1, d_loss, 100.0 * accuracy, g_loss))
        sample_images(gerador, iteration) 

def sample_images(generator,iteration, image_grid_rows=4, image_grid_columns=4):

  z = np.random.normal(0,1,(image_grid_rows * image_grid_columns, noise_input_dim))
  
  gen_imgs = generator.predict(z)

  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(image_grid_rows, image_grid_columns, figsize=(200,200), sharey=True, sharex=True)

  cnt = 0

  for i in range(image_grid_rows):
    for j in range(image_grid_columns):
      axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap='gray')
      axs[i,j].axis('off')
      cnt += 1 
  plt.savefig('/content/Drive/MyDrive/treinamento_gan3/'+str(iteration)+'.jpg')



In [ ]:
iterations = 9000
batch_size = 32
sample_interval = 500

train(iterations, batch_size, sample_interval)

A saída de streaming foi truncada nas últimas 5000 linhas.
1/1 [==============================] - 0s 26ms/step
2000 [D loss: 0.193329, acc.: 93.75%] [G loss: 4.384587]
1/1 [==============================] - 0s 44ms/step
2500 [D loss: 0.047785, acc.: 98.44%] [G loss: 3.338649]
1/1 [==============================] - 0s 26ms/step
3000 [D loss: 0.072351, acc.: 98.44%] [G loss: 5.119078]
1/1 [==============================] - 0s 26ms/step
3500 [D loss: 0.197468, acc.: 95.31%] [G loss: 3.041004]
1/1 [==============================] - 0s 32ms/step
4000 [D loss: 0.408315, acc.: 82.81%] [G loss: 3.064930]
1/1 [==============================] - 0s 26ms/step
